In [1]:
import numpy as np
import pandas as pd

#### the training_split_by_us transaction records info
historical transaction records:

[ training=14441899, unique cardid=161533 ]

[ testing=3607915, unique cardid=40384 ]

new merchant transaction records:

[ training=995721, unique cardid=161533 ]

[ testing=249641, unique cardid=40384 ]

#### merchant information tb
includes categorical variable transforms to dummy variables

In [3]:
# the 41 chosen records already been transformed
chosen=pd.read_csv('the_chosen_41_merchant_id_that_had_duplicated_records.csv')
merchant_tb=pd.read_csv('merchants.csv')


#transform data tyep for cg data and apply transformation
merchant_tb[['category_1', 'category_2', 'category_4','most_recent_sales_range', 'most_recent_purchases_range','active_months_lag3','active_months_lag6','active_months_lag12']]=merchant_tb[['category_1', 'category_2', 'category_4','most_recent_sales_range', 'most_recent_purchases_range','active_months_lag3','active_months_lag6','active_months_lag12']].astype({'category_1':'category', 'category_2':'category', 'category_4':'category','most_recent_sales_range':'category', 'most_recent_purchases_range':'category','active_months_lag3':'category','active_months_lag6':'category','active_months_lag12':'category'})

merchant_tb.category_2.replace(np.nan,999,inplace=True)

In [4]:
dlst=chosen['Unnamed: 0'].tolist()
d41_id=merchant_tb.set_index(['merchant_id']).loc[dlst].groupby('merchant_id').first()
merchant=merchant_tb.set_index(['merchant_id']).drop(dlst,axis=0)
merchant=pd.concat([merchant,d41_id],axis=0).reset_index()

In [5]:
merchant_n=merchant[['merchant_id', 'merchant_group_id', 'merchant_category_id','numerical_1', 'numerical_2','avg_sales_lag3', 'avg_purchases_lag3','avg_sales_lag6', 'avg_purchases_lag6','avg_sales_lag12', 'avg_purchases_lag12']]

merchant_c=pd.get_dummies(merchant[['category_1','category_2','category_4','most_recent_sales_range', 'most_recent_purchases_range',
                    'active_months_lag3','active_months_lag6','active_months_lag12']])

merchant_info=pd.concat([merchant_n,merchant_c],axis=1)

#29

In [14]:
#these 13 records all have nan in avg_sales_lag3,avg_sales_lag6,avg_sales_lag12
#merchant_tb[merchant_tb.avg_sales_lag3.isnull()].isnull().any()

In [6]:
def find_mean_value(tb,null_list):
    value=[]
    for i in null_list:
        value.append(tb['mean'].loc[i])
    return value

In [7]:
# find the value from 'avg_sales_lag3','avg_sales_lag6','avg_sales_lag12' that can replace null the value is obtained by grouping the merchant category_id
null_tb=merchant_info[merchant_info.isnull().any(1)]
null_lst=null_tb.merchant_group_id.tolist()
avg_sales_lag3=find_mean_value(merchant_info.groupby('merchant_group_id')['avg_sales_lag3'].agg(['mean']),null_lst)
avg_sales_lag6=find_mean_value(merchant_info.groupby('merchant_group_id')['avg_sales_lag6'].agg(['mean']),null_lst)
avg_sales_lag12=find_mean_value(merchant_info.groupby('merchant_group_id')['avg_sales_lag12'].agg(['mean']),null_lst)
null_tb.drop(['avg_sales_lag3','avg_sales_lag6','avg_sales_lag12'],axis=1,inplace=True)
avg_sales=pd.DataFrame([avg_sales_lag3,avg_sales_lag6,avg_sales_lag12]).T
avg_sales.columns=['avg_sales_lag3','avg_sales_lag6','avg_sales_lag12']

C:\Anacon\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
#replace the inf value with the mean of the same group
inf_tb=pd.concat([null_tb.reset_index(drop=True),avg_sales],axis=1).set_index("merchant_group_id")
inf_lst=inf_tb.index[inf_tb.isin([np.inf]).any(1)].tolist()
avg_purchases_lag3=find_mean_value(merchant_info.replace([np.inf, -np.inf], np.nan).groupby('merchant_group_id')['avg_purchases_lag3'].agg(["mean"]),inf_lst)
avg_purchases_lag6=find_mean_value(merchant_info.replace([np.inf, -np.inf], np.nan).groupby('merchant_group_id')['avg_purchases_lag6'].agg(["mean"]),inf_lst)
avg_purchases_lag12=find_mean_value(merchant_info.replace([np.inf, -np.inf], np.nan).groupby('merchant_group_id')['avg_purchases_lag12'].agg(["mean"]),inf_lst)

avg_purchases=pd.DataFrame([avg_purchases_lag3,avg_purchases_lag6,avg_purchases_lag12]).T
avg_purchases.columns=['avg_purchases_lag3','avg_purchases_lag6','avg_purchases_lag12']

In [9]:
inf_tb.loc[inf_lst]=pd.concat([(inf_tb.loc[inf_lst].drop(['avg_purchases_lag3','avg_purchases_lag6','avg_purchases_lag12'],axis=1).reset_index()),avg_purchases],axis=1).set_index('merchant_group_id')
inf_tb=inf_tb.reset_index()
inf_tb1=inf_tb[['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'numerical_1', 'numerical_2', 'avg_sales_lag3', 'avg_purchases_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'avg_sales_lag12',
       'avg_purchases_lag12', 'category_2', 'active_months_lag3',
       'active_months_lag6', 'active_months_lag12', 'category_1_N',
       'category_1_Y', 'category_4_N', 'category_4_Y',
       'most_recent_sales_range_A', 'most_recent_sales_range_B',
       'most_recent_sales_range_C', 'most_recent_sales_range_D',
       'most_recent_sales_range_E', 'most_recent_purchases_range_A',
       'most_recent_purchases_range_B', 'most_recent_purchases_range_C',
       'most_recent_purchases_range_D', 'most_recent_purchases_range_E']]

In [10]:
merchant_no_null_inf=merchant_info.drop(merchant_info[merchant_info.isnull().any(1)].index).reset_index(drop=True)
merchant_final=pd.concat([inf_tb1,merchant_no_null_inf],axis=0)

In [20]:
#merchant_final.to_csv("merchant_final.csv")

#### combine the his and new merchant transaction tb (because some cardID don't have new merchant)

In [2]:
hist_tran=pd.read_csv('C:/Users/Rai Chiang/Documents/Depaul Course/672/data/split/training_split_by_us_transaction.csv')
len(hist_tran)#14441899

new_tran=pd.read_csv('C:/Users/Rai Chiang/Documents/Depaul Course/672/data/split/testing_split_by_us_transaction.csv')

In [3]:
# category 2 and 3 still have nan, replace them with 999
#hist_tran.isnull().any()

#len(hist_tran.category_3[hist_tran.category_3.isnull()]) #nan 88309
#hist_tran.category_3.value_counts()
hist_tran.category_3.replace(np.nan,999,inplace=True)

#len(hist_tran.category_2[hist_tran.category_2.isnull()]) #nan 1311964
#hist_tran.category_2.value_counts()
hist_tran.category_2.replace(np.nan,999,inplace=True)
#14441899

#hist_tran.installments.value_counts()
hist_tran.installments.replace(-1,999,inplace=True)

In [4]:
#new_tran.isnull().any()

#there 17489 records are all null except for card-id 
#len(new_tran[new_tran.authorized_flag.isnull()])

#the new merchant records id list that has null values
no_value_record=new_tran.card_id[new_tran.authorized_flag.isnull()].tolist()

#drop 17489 records that has no values
new_tran=new_tran.set_index('card_id').drop(no_value_record,axis=0).reset_index()

In [5]:
#after removing the null records found that category 2 and 3 still have nan, replace them with 999
#new_tran.isnull().any()

#len(new_tran.category_3[new_tran.category_3.isnull()]) #nan 27655
#new_tran.category_3.value_counts()
new_tran.category_3.replace(np.nan,999,inplace=True)

len(new_tran.category_2[new_tran.category_2.isnull()]) #nan 55048
#new_tran.category_2.value_counts()
new_tran.category_2.replace(np.nan,999,inplace=True)
#978232


In [6]:
trans_tb=pd.concat([hist_tran,new_tran],axis=0)

In [ ]:
trans_tb=trans_tb.sort_values(by="card_id")

#### transaction + merchant info
categorical variables in transaction transforms to dummy variables
then the whole historical tb merged with the corresponding merchant info

In [8]:
#trans_tb.dtypes
#transform data type and apply transformation
trans_tb[['authorized_flag','month_lag','installments','category_1','category_2','category_3']]=trans_tb[['authorized_flag','month_lag','installments','category_1','category_2','category_3']].astype({'authorized_flag':'category','month_lag':'category','installments':'category','category_1':'category','category_2':'category','category_3':'category'})
trans_tb_cg=pd.get_dummies(trans_tb[['authorized_flag','month_lag','installments','category_1','category_2','category_3']])
#45



In [9]:
#create df for trans_tb
t_tb=pd.concat([trans_tb[['card_id','merchant_id','merchant_category_id','purchase_amount']],trans_tb_cg],axis=1)
# 49 columns

#t_tb.to_csv('Elo Project [feature creation]Trans+newTrans_part1.csv')

#### import the transformed transaction files

In [2]:
chunks=[]
for chunk in pd.read_csv('C:/Users/Rai Chiang/Documents/Depaul Course/672/data/Elo Project [feature creation]/Elo Project [feature creation]Trans+newTrans_part1.csv',chunksize=50000,index_col=['Unnamed: 0']):
    chunks.append(chunk)

In [4]:
#numerical variable
def numerical(groupby_tb,column,method):
    if method=="m":
        mean=groupby_tb[column].agg(['mean']).reset_index().groupby(['card_id']).agg(['mean'])[column] #mean of the mean of the merchant record
        max_=groupby_tb[column].agg(['max']).reset_index().groupby(['card_id']).agg(['max'])[column] #merchant record max
        min_=groupby_tb[column].agg(['min']).reset_index().groupby(['card_id']).agg(['min'])[column] #merchant record min
        tb=pd.concat([pd.concat([mean,max_],axis=1),min_],axis=1)

    if method=="s":
        tb=groupby_tb[column].agg(['std'])#merchant record min
        
    
    return tb

In [4]:
def transation_aggregation(ck):

    #seperte the numerica, categoical variable and index
    var=ck.columns.tolist()
    numerical_var=['purchase_amount']
    trans_info_var=['card_id','merchant_id','merchant_category_id']
    categorical_var=[ i for i in var if i not in numerical_var and i not in trans_info_var]

    #63 categorical columns into one row
    trans_merchant_cg=ck.groupby(['card_id'])[categorical_var].agg(['sum']).reset_index().groupby(['card_id']).agg(['sum'])
    trans_merchant_cg.columns=categorical_var

    tb_grouped1=ck.groupby(['card_id'])
    unique_merchant=tb_grouped1[['merchant_id','merchant_category_id']].agg(['nunique'])
    trans_reocord=tb_grouped1[['merchant_id']].agg(['count'])
    trans_info=trans_reocord.join(unique_merchant)
    trans_info.columns=['# of trans_reocords','# of unique_merchant','# of unique_merchant_category_transaction']

    #numerical var into one row
    tb_grouped2=ck.groupby(['card_id','merchant_category_id'])
    tb_grouped3=ck.groupby(['card_id'])

    trans_merchant_nm=pd.concat([numerical(tb_grouped2,numerical_var,method='m'),numerical(tb_grouped3,numerical_var,method='s')],axis=1)

    #merge transinfo + numerical var + categorical var
    final_tb=trans_info.join(trans_merchant_nm).join(trans_merchant_cg)
    
    return final_tb 

In [5]:
for i in range(len(chunks)):
    if i ==0:
        tb=transation_aggregation(chunks[i])
    else:
        tb=pd.concat([tb,transation_aggregation(chunks[i])],axis=0)
    

C:\Anacon\lib\site-packages\pandas\core\base.py:331: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return self.obj.drop(self.exclusions, axis=1)


In [6]:
tb.isnull().any()

# of trans_reocords                          False
# of unique_merchant                         False
# of unique_merchant_category_transaction    False
(purchase_amount, mean, mean)                False
(purchase_amount, max, max)                  False
(purchase_amount, min, min)                  False
(purchase_amount, std)                        True
authorized_flag_N                            False
authorized_flag_Y                            False
month_lag_-13.0                              False
month_lag_-12.0                              False
month_lag_-11.0                              False
month_lag_-10.0                              False
month_lag_-9.0                               False
month_lag_-8.0                               False
month_lag_-7.0                               False
month_lag_-6.0                               False
month_lag_-5.0                               False
month_lag_-4.0                               False
month_lag_-3.0                 

In [20]:
tb[tb[('purchase_amount', 'std')].isnull()] # for these 5 ids they only have 1 trans record, so the std of numeric is nan

,# of trans_reocords,# of unique_merchant,# of unique_merchant_category_transaction,"(purchase_amount, mean, mean)","(purchase_amount, max, max)","(purchase_amount, min, min)","(purchase_amount, std)",authorized_flag_N,authorized_flag_Y,month_lag_-13.0,...,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_999.0,category_3_999,category_3_A,category_3_B,category_3_C
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_4d52025e51,1,1,1,-0.730574,-0.730574,-0.730574,NaN,0,1,0,...,0,0,0,0,1,0,0,1,0,0
C_ID_590ef8c490,1,1,1,-0.733384,-0.733384,-0.733384,NaN,0,1,0,...,1,0,0,0,0,0,0,0,1,0
C_ID_750c9b63cc,1,1,1,-0.676989,-0.676989,-0.676989,NaN,0,1,0,...,0,0,0,0,1,0,0,1,0,0
C_ID_7bbe08db78,1,1,1,-0.728876,-0.728876,-0.728876,NaN,0,1,0,...,0,0,0,0,0,1,0,0,1,0
C_ID_bbcf340c7b,1,1,1,-0.457197,-0.457197,-0.457197,NaN,0,1,0,...,0,0,0,0,1,0,0,1,0,0
C_ID_c36b5faa2f,1,1,1,-0.730078,-0.730078,-0.730078,NaN,0,1,0,...,1,0,0,0,0,0,0,1,0,0
C_ID_ef7da73fc5,1,1,1,-0.375783,-0.375783,-0.375783,NaN,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [20]:
#len(tb) #161840
#len(tb.index.unique()) #161533
#tb.to_csv('Elo Project [feature creation]Trans+newTrans_part2.csv')

#the card id that appears in two chunks
db_card_ids=tb[pd.Series(tb.index).value_counts()>1].index.unique().tolist()

len(db_card_ids)
#np.save('db_card_ids.npy',db_card_ids)

C:\Anacon\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


307

In [ ]:
#db_card_ids=np.load('db_card_ids.npy')
#db_trans_records=t_tb.set_index('card_id').loc[db_card_ids]
#db_trans_records.to_csv('Elo Project [feature creation]cardIDS_307.csv')

#cardIDS_307=transation_aggregation(db_trans_records)

In [6]:
#tb=pd.read_csv('Elo Project [feature creation]Trans+newTrans_part2.csv')

In [13]:
cardIDS_307.set_index(['card_id'],inplace=True)

In [17]:
cardIDS_307.columns=tb.columns.tolist()

In [21]:
tb.drop(db_card_ids,axis=0,inplace=True)

In [24]:
transaction_tb=pd.concat([tb,cardIDS_307],axis=0)
#transaction_tb.to_csv('Elo Project [feature creation] cardInfo + transactions_transoform to one row.csv')
#161533

In [26]:
transaction_tb.head()

,# of trans_reocords,# of unique_merchant,# of unique_merchant_category_transaction,"(purchase_amount, mean, mean)","(purchase_amount, max, max)","(purchase_amount, min, min)","(purchase_amount, std)",authorized_flag_N,authorized_flag_Y,month_lag_-13.0,...,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_999.0,category_3_999,category_3_A,category_3_B,category_3_C
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,151,31,19,-0.442338,1.507069,-0.728876,0.296638,35.0,116.0,0,...,1.0,0.0,121.0,0.0,1.0,28.0,0,0.0,127.0,24
C_ID_0001506ef0,67,29,20,-0.659388,1.493545,-0.740491,0.466421,4.0,64.0,2,...,2.0,0.0,66.0,0.0,0.0,0.0,0,67.0,1.0,0
C_ID_000183fdda,155,84,38,-0.378612,2.764788,-0.737892,0.508951,7.0,148.0,0,...,7.0,1.0,142.0,0.0,1.0,4.0,5,0.0,109.0,41
C_ID_00027503e2,42,12,7,-0.739320,-0.734135,-0.745871,0.003334,18.0,24.0,0,...,3.0,0.0,39.0,0.0,0.0,0.0,0,42.0,0.0,0
C_ID_0002c7c2c1,73,44,25,-0.652780,-0.296112,-0.737892,0.079058,9.0,64.0,0,...,73.0,0.0,0.0,0.0,0.0,0.0,0,73.0,0.0,0


In [14]:
t=pd.read_csv('Elo Project [feature creation] cardInfo + transactions_transoform to one row.csv')

C:\Anacon\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### merge multiple merchant info to one sigle row belong to same card ID

In [3]:
merchant_final=pd.read_csv('merchant_final.csv',index_col=['Unnamed: 0'])

In [5]:
def merchant_aggrgation(tb):

    var=tb.columns.tolist()

    #seperte the numerica, categoical variable and index
    numerical_var=['numerical_1','numerical_2','avg_purchases_lag3','avg_purchases_lag6','avg_purchases_lag12',
                  'avg_sales_lag3','avg_sales_lag6','avg_sales_lag12']
    info_var=['card_id','merchant_id','merchant_category_id','merchant_group_id']
    categorical_var=[ i for i in var if i not in numerical_var and i not in info_var]
    
    #categorical columns into one row
    trans_merchant_cg=tb.groupby(['card_id'])[categorical_var].agg(['sum']).reset_index().groupby(['card_id']).agg(['sum'])

    #transaction info into one row
    tb_grouped1=tb.groupby(['card_id'])
    trans_info=tb_grouped1[['merchant_group_id']].agg(['nunique'])
    trans_info.columns=['# of unique_merchant_group']
    
    #numerical var into one row
    tb_grouped2=tb.groupby(['card_id','merchant_group_id','merchant_category_id'])

    tb_grouped3=tb.groupby(['card_id'])

    trans_merchant_nm=pd.concat([numerical(tb_grouped2,numerical_var,method='m'),numerical(tb_grouped3,numerical_var,method='s')],axis=1)

    final_tb=trans_info.join(trans_merchant_nm).join(trans_merchant_cg)
    
    return final_tb
    

In [6]:
merchant_records=[]
for i in range(len(chunks)):
    merchant_records.append(merchant_aggrgation(pd.merge(chunks[i][['card_id','merchant_id']],merchant_final,how="left",on='merchant_id')))

C:\Anacon\lib\site-packages\pandas\core\base.py:331: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return self.obj.drop(self.exclusions, axis=1)
C:\Anacon\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 3 on the right)
  warnings.warn(msg, UserWarning)


In [7]:
merchant_record_tb=merchant_records[0]
for i in merchant_records[1:]:
    merchant_record_tb=pd.concat([merchant_record_tb,i],axis=0)
    
#161840

In [10]:
merchant_record_tb.head()

,# of unique_merchant_group,"(numerical_1, mean, mean)","(numerical_2, mean, mean)","(avg_purchases_lag3, mean, mean)","(avg_purchases_lag6, mean, mean)","(avg_purchases_lag12, mean, mean)","(avg_sales_lag3, mean, mean)","(avg_sales_lag6, mean, mean)","(avg_sales_lag12, mean, mean)","(numerical_1, max, max)",...,"(most_recent_sales_range_A, sum, sum)","(most_recent_sales_range_B, sum, sum)","(most_recent_sales_range_C, sum, sum)","(most_recent_sales_range_D, sum, sum)","(most_recent_sales_range_E, sum, sum)","(most_recent_purchases_range_A, sum, sum)","(most_recent_purchases_range_B, sum, sum)","(most_recent_purchases_range_C, sum, sum)","(most_recent_purchases_range_D, sum, sum)","(most_recent_purchases_range_E, sum, sum)"
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,15,1.648884,1.622445,1.413299,1.544300,1.596003,1.194400,1.267667,1.310867,15.855951,...,51.0,12.0,14.0,61.0,13.0,44.0,17.0,12.0,64.0,14.0
C_ID_0001506ef0,19,1.660201,1.558317,1.483648,1.633043,1.683127,1.274138,1.349655,1.394138,15.389950,...,24.0,12.0,16.0,12.0,3.0,27.0,7.0,18.0,12.0,3.0
C_ID_000183fdda,64,0.176497,0.156788,1.331406,7.459736,8.055006,1.305955,3.272378,3.532907,4.523215,...,7.0,51.0,49.0,29.0,19.0,6.0,42.0,52.0,33.0,22.0
C_ID_00027503e2,7,0.105299,0.097863,1.966788,2.260121,2.279319,1.487500,1.645833,1.652500,0.864615,...,0.0,4.0,22.0,11.0,5.0,0.0,2.0,33.0,4.0,3.0
C_ID_0002c7c2c1,34,5.981060,5.638288,1.396683,2.909823,4.358277,1.193452,2.347798,3.596845,117.830742,...,35.0,8.0,9.0,6.0,15.0,35.0,7.0,9.0,13.0,9.0


In [9]:
len(merchant_record_tb) #161840

161840

In [10]:
len(merchant_record_tb.index.unique()) #unique id, 161533-161840=307, 307 of card_id has duplicate record due to the split of chunks

161533

In [17]:
db_trans_records=pd.read_csv('Elo Project [feature creation]cardIDS_307.csv')
len(db_trans_records)#307

#db_merchant_info=merchant_aggrgation(pd.merge(db_trans_records[["card_id","merchant_id"]],merchant_final,how="left",on='merchant_id'))


307

In [215]:
merchant_record_tb.drop(db_trans_records.card_id.unique().tolist(),axis=0,inplace=True)
len(merchant_record_tb) #161226 =>  161533-307=161226

merchant_record_tb=pd.concat([merchant_record_tb,db_merchant_info],axis=0)
len(merchant_record_tb)==len(merchant_record_tb.index.unique()) #True, make sure all the rows are unique cardID

#merchant_record_tb.to_csv('Elo Project [feature creation] cardInfo + merchantInfo_transofrm to one row.csv')
#merchant_record_tb.head() #161533